In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

i,j=11,3
ds2018 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
ds2019 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2019.nc")

ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')

ds[['nbart_red','nbart_green','nbart_blue']].clip(0,500).to_array().plot.imshow(col='time', col_wrap=6)

In [ ]:
#ds.where((ds.nbart_blue - ds.nbart_blue.quantile(0.25, dim='time'))<200).isel(time=(np.count_nonzero(~np.isnan(ds.nbart_blue.values), axis=(1,2)))>400*300)#.rolling(time=7, min_periods=3, center=True).median()[['nbart_red','nbart_green','nbart_blue']].clip(0,500).to_array().plot.imshow(col='time', col_wrap=6)
fds = ds.where((ds.nbart_blue - ds.nbart_blue.quantile(0.25, dim='time'))<500)
fds.isel(time=(np.count_nonzero(~np.isnan(fds.nbart_blue.values), axis=(1,2)))>400*300).rolling(time=7, min_periods=3, center=True).median()[['nbart_red','nbart_green','nbart_blue']].clip(0,500).to_array().plot.imshow(col='time', col_wrap=6)

In [ ]:
ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')
ds = ds.where((ds.nbart_blue - ds.nbart_blue.quantile(0.25, dim='time'))<500)
#ds = ds.isel(time=(np.count_nonzero(~np.isnan(ds.nbart_blue.values), axis=(1,2)))>400*200)
ds.nbart_blue.rolling(time=3, min_periods=2, center=True).var().plot(col='time', col_wrap=6, robust=True)

In [ ]:
(ds.nbart_blue - ds.nbart_blue.quantile(0.25, dim='time')<200).plot(col='time', col_wrap=6, robust=True)

In [ ]:
from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from sklearn.decomposition import PCA

def generate_bmask(ds):
    blue = ds.nbart_blue.astype(np.float32) / 1e4

    # 1.- Remove images with less than 1000 valid pixels
    blue = blue.isel(time=(np.count_nonzero(~np.isnan(blue.values), axis=(1,2)))>1000)

    # 2.- Create mask for reflectances with deviations more than 10% from lower quartile
    # 2.- Create mask for reflectances with deviations more than 0.1 from lower quartile
    
    threshold = (blue.quantile(0.25, dim='time')*0.5).clip(min=0.05)
    
    #bmask = ((blue - blue.quantile(0.25, dim='time'))>.1).values
    bmask = ((blue - blue.quantile(0.25, dim='time'))>threshold).values

    # 3.- Remove small objects (< 25 pixels) and grow a 9 pixel disk buffer around remaining objects
    for i in range(bmask.shape[0]):
        bmask[i] = remove_small_objects(bmask[i], 25)
        bmask[i] = dilation(bmask[i], disk(9))
        bmask[i][np.isnan(blue[i].values)] = False
        
    # 4.- Wipe frames with more than 50% missing pixels
    #bmask[(np.count_nonzero(bmask, axis=(1,2)) / 160000)>.5] = np.nan

    return bmask

bmask = generate_bmask(ds)

ds[['nbart_red','nbart_green','nbart_blue']].where(~bmask).clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
blue = ds.nbart_blue.astype(np.float32) / 1e4

blue.isel(time=3).plot()

In [ ]:
blue.quantile(0.25, dim='time').plot()

In [ ]:
ds.nbart_blue.where(~bmask).isel(time=3).plot()